# Labsheet 6: Odometry

This worksheet will cover reading the Rotary Encoders on your 3Pi+ robot, and then using this sensor data to maintain an estimate of the robot position on a 2D plane.  

In previous labsheets you will have:
- blah
- blah

In this labsheet we are going to:



## A note on Power  vs. Speed

In the previous labsheet we sent **power** values to the motors, and we observed the **speed**.   Note that, we didn't set or control the speed.  

**It is important to recognise that the Romi robot was not controlling speed**, and it could not know what speed the wheels were rotating at.  Consider, if you applied resistence to the wheel, it slowed down.  To achieve a constant speed, your Romi would need to increase the power to the motor when the wheels are meeting resistance.  This would be the essence of a closed-loop, feedback control system.

In order to begin to autonomously control speed, your Romi must be able to detect the rotation of the wheels with respect to time.  This lab sheet investigates a rotary encoder sensor for detecting rotation (distance).  Labsheet 7 concerns time, and will allow you to calculate and control speed.


<hr><br><br><br><br>

# Odometry

**`Odometry`** refers to maintaining an estimate of position using sensors.  The 3Pi+ has a rotary encoder per motor, which allows for the detection of rotation.  By using a last-known position and a subsequent measurement of change, it is possible to calculate an estimate of the robot position.  This technique is known as **`dead reckoning`**.

<p align="center">
<img width="75%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_InfoSource.png?raw=true">
</p>

In the last Labsheet "Line Following", you will have developed your first `closed-loop` control system.  Hopefully, your robot was able to follow the line sufficiently well.  This would mean your robot appeared to have an intelligent autonomous behaviour.  It is important to recognise that to achieve this, your robot was utilising a `source of information` from the environment - the line.  

In other parts of the line following challenge, this same information is not available.  For instance:
- traversing the gap in the line.
- determining the end of the line.
- returning to home (start position). 

If we can utilise the rotary encoders to estimate the robot position, we can use this information to perform operations such as:
- turn to a specific angle
- move to a specific location
- maintain a straight course of travel
- estimate the distance travelled
- etc.


<hr><br><br><br><br>

# Encoders

The 3Pi+ has a sensor subsystem called a `rotary encoder`, and one of these attached to each motor (two in total).   The rotary encoder is able to indicate when rotation has occured.  Some rotary coders are `absolute`, meaning they can be read to know the exact rotation of the device they are attached to.  Other encoders are `incremental`, meaning that when read they only indicate a change in rotation. 

There are two principle ways to read an encoder:
- **`polling`**: if a device is polled, it means the device is read at regular time intervals.  It is important that the read operation happens sufficiently frequently or important information may be missed.  This technique will be demonstrated and discussed as insufficient in one of the unit lectures.
- **`interrupt-driven`**: in interrupt driven subsystems, either the subsystem can issue a flag to indicate new information is ready, or the host computer can set peripheral hardware to monitor for change.  In both cases, an `interrupt` flag (request) is set, and an `interrupt service routine` (IRS) is required to handle the information within an appropriate time.

The 3Pi+ 32u4 microcontroller is able to configure peripheral hardware to monitor for status changes on some of the `gpio` pins. Every time an encoder is moved, it will trigger the `interrupt service routine` (ISR).  The ISR will effectively keep a count of how many times it has been called.  The ISR will also register whether it was a forwards or backwards movement, to increment or decrement the count respectively:

<p align="center">
<br>
<img width="50%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderCount.png?raw=true">
<br>
</p>

## Exercise 1: 3Pi+ Encoders

1. Does the 3Pi+ have `absolute` or `incremental` rotary encoders?  (documentation <a href="https://www.pololu.com/docs/0J83/5.4">1</a>, <a href="https://www.pololu.com/product/3081">2</a>)

2. We could say that the encoders detect the rotation of either the motors (`actuators`), or the wheels (`effectors`).  Which of these (motors/wheels) is the more important description of the nature of the information the 3Pi+ will be able to perceive?

3. The designers of the 3Pi+ attached the encoders to the motors prior to the reduction gearbox.  
  - What is an advantage of this design choice?  
  - What is a disadvantage of this design choice?


<hr><br><br><br><br>

# Quadrature Encoding

For this section we will consider only 1 encoder, which has 2 `hall effect` sensors (`transducers`), and 1 magnet.

<p align="center">
<br>
<img width="75%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderAssembly.png?raw=true">
<br>
</p>

When the wheel is rotated, the magnet is rotated, and so the magnetic field moves over the sensor circuit.  The two `Hall effect` sensors alter their output with the magnetic field, and so two signals are generated.  The state of these two signals can be interpretted as two logic signals.  The combination of two logic signals (either `0` or `1`) provide 4 possible states (base two, two signals: $2^2$), hence the name `quadrature` encoders. 

The two logic signals are typically referred to as **Channel A** and **Channel B**.  Importantly, the **relationship** between these two signals is defined by their physical placement on the sensor circuit.  The relationship between Channel A and B will inform the measurement of rotation.
> "What is meant by relationship?"

Here, the relationship of the two signal means `how they change with respect to one another`.  If they were `independent` variables, it would mean there was no relationship at all, and they could vary independently.  In the case of this rotary encoder, both of the `Hall effect` sensors are exposed to the same varying magnetic field, and both of their positions relative to the field are fixed by the construction of the circuit board.  This means that the Channel A and Channel B signals can be understood to vary relative to one another.  

<p align="center">
<br>
<img width="75%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_HallEffect90.png?raw=true">
<br>
</p>

The graphic above is for a <a href="https://www.pololu.com/product/3081">similar sensor circuit</a> as built into the 3Pi+ encoder subsystem.  The graphic has been annotated to indicate where the `rear motor shaft` is located, which is the point of rotation of the magnet.  The two `Hall effect` sensors are annotated, and their position is marked to show that they sit at 90&deg; to one another with respect to the origin of the rotating magnetic field.  This means that their measurement of the magnetic field will **always** be 90&deg; out of phase (out of synchrony, or shifted, by a quarter cycle).

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_90Phase.png?raw=true">
<br>
</p>

The above diagram illustates how the magnetic field (north and south poles, red and blue respectively) will move over the two `Hall effect` sensors.  A small red dot has been added to help indicate how the field is rotating across the four states in time, read left to right respectively.  In reality, the magnet will move continuously, but these are the essential 4 states registered by the encoder.

The logic signals for Channel A and B are visible above as  the `square waveforms`, and coloured to match the magnetic field polarity detected in each state.  These waveforms will endlessly repeat their high-low-high transitions as the magnet is rotated above them.  Notice that, if we mark in time where each Channel transitions to logic `1` (or high), we can see they are 90&deg; out of phase with respect to each other.

|Channel B   | Channel A   |
|:---:|:---:|
| 0  | 0  |
| 0  | 1  |
| 1  | 1  |
| 1  | 0  |

We can examine the above diagram and produce the above logic table, where south-polarity (blue) is logic 0, and north-polarity (red) is logic 1.  These are the 4 possible states of the `hall effect` sensors combined.  If the magnet continues rotating in the same direction as the illustration, this logic table would repeat itself.  At this point, we could be satisfied by registering a logic change, to keep a "count" of how many times the encoder has detected a rotation.  However, we know that the wheels of the 3Pi+ can be rotated in either direction.

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderXORSignals.png?raw=true">
<br>
</p>

It is therefore important to register the `logic transitions` in the signals.  In the above illustration, the transitions of the waveforms have been annotated.  Note that, the transitioning logic has an arrow next to it - this is because the progression of these waveforms could be read left-to-right, or right-to-left, depending on which way the wheel is rotating.  

In order to capture a transition from low-to-high, or high-to-low, it is necessary to have `prior` information (what was the state of the channel before?).  

<p align="center">
<br>
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderDirection.png?raw=true">
<br>
</p>

In the above illustration, the `prior` state of the encoder is marked in red, as `A = 1, B = 1`.  Our code would then need to work out if it is either **channel A** which changes, or **channel B** which changes, away from the prior state.  This "direction" of this transition away from the prior state will allow our robot to count up or down with respect to the direction of travel:

<p align="center">
<br>
<img width="50%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/3PI_EncoderCount.png?raw=true">
<br>
</p>





<hr><br><br><br><br>

# Pin Change Interrupts

In the previous section, we have discussed how the 3Pi+ can determine direction of rotation by registering the transition of the A and B channels of the encoder.  We have used the assumption that an encoder `interrupt service routine` (ISR) will activate when the wheel is moved.  This is achieved on the 3Pi+ by using a **`Pin Change Interrupt`**.  

**`Pin Change Interrupts`** are a special piece of hardware.  Microcontrollers have many useful built-in peripherals.  A pin change interrupt watches a specific pin on the chip for a change of state (low to high, or high to low).  When the microcontroller detects a change, it can be set to automatically run a piece of code.  The piece of code which runs automatically is called an **`Interrupt Service Routine`** (ISR).  An ISR is not exclusively for pin changes - they can be triggered to run by many peripherals, taking input internal or external to the microcontroller.  

An `interrupt` is so called because it will stop (pause) sequence of your main code.  These are especially useful to catch external events, rather than `polling` to detect them.  However, most microcontrollers have only one central processing unit (CPU, often called a 'core').  Therefore, because your code within `loop()` runs on the only CPU inside the 32u4, it will be paused to run an ISR instead.  

You can think of an ISR as being a background task which runs "only when it needs to".  However, you are responsible for determining the cause of the interrupt (what triggers it), and the efficiency of the ISR (how much CPU time it will use).  

> As a general rule, code within ISR's should be as short as possible in order to avoid taking up too much CPU time.

If you are experimenting with an ISR and your robot starts to behave in a strange way, you may have made your ISR too big (or more seriously, have a bug within the ISR code).  Note that, using ISR's means the execution of your code is no longer `deterministic` - the compiler cannot predict when an ISR will activate, and neither can you.  For example, we cannot know when a wheel and motor will rotate.  

Obviously, a Pin Change Interrupt is very convenient for watching the state of the encoders.  We can consider catching encoder changes to be a high priority, perhaps higher than our main program.  For example, if we miss encoder changes, we will lose track of our robot position.  

## A Slight Complication

A limitation is that only specific pins can be monitored by a Pin Change Interrupt, and there are a finite number of pins to use.  The 3Pi+ has two encoders, each with two pins.  This means that in an ideal world we might like to watch 4 independent pin changes in total.  

The designers of 3Pi+ decided to use some electronics to use just 1 pin per encoder to trigger a pin change interrupt - using just one pin to register a change for either pins to an encoder.  They have achieved this by using an electronic XOR (eXclusive OR) logic gate across the A and B channels, one XOR for each encoder.  

Recall that the signals for A and B are always out of phase by 90&deg;.  Therefore, by XOR'ing A with B, we can register when a change occurs to either A or B through just one pin (it will always be exclusively one or the other that changed).  

This electronic XOR result is sent through a single Pin Change Interupt pin, whilst the normal result for the channel B encoder pin can be read with a normal digitalRead().  By using a software-XOR operation to compare the pin change interrupt source and Channel B, the state of Channel A can be identified.  You can find more information in the <a href="https://www.pololu.com/docs/0J83/5.4">documentation</a>

**For now, you don't need to fully understand this operation - it is a matter of interest**.  If you want to fully understand it, I suggest you draw out a logic truth table for XOR, then some pin transitions, and apply XOR logic.  In any case, you'll notice the following section of code within our ISRs to perform this XOR and restore the A and B pin states.  It looks a little strange - **but don't worry, it is complete and you do not need to modify it:**

```c
  // We know that the ISR is only called when a pin changes.
  // We also know only 1 pin can change at a time.
  // The XOR(AB) signal change from "Channel A" triggers ISR.
  
  // First, Read in the new state of the encoder pins.
  // Standard pins, so standard read functions.
  boolean newE1_B = digitalRead( E1_B_PIN ); // normal B state
  boolean newE1_A = digitalRead( E1_A_PIN ); // XOR(AB)

  // Software XOR (^) logically infers
  // the true value of A given the state of B
  newE1_A = newE1_A ^ newE1_B;
```
**You can just assume the above XOR operation works, and simply make sure you do not delete it from the template code provided.**





# Setting Up Pin Change Interrupts

Configuring the microcontroller hardware to use the Pin Change Interrupts is a little bit complicated, so we have done this for you.  For further information you can look at the two functions:
- **setupEncoder0()**
- **setupEncoder1()** 

These set up Pin Change Interrupts for the two encoders on the  3Pi+.  Both functions are heavily commented.  Both involve bitwise operations to registers within the microcontroller.  Setting bits in registers activates peripherals or configures the microcontroller - this should have been covered in a lecture you have received.  If you'd like to experiment with similar operations, it is recommended you look at the labsheet concerning the Timer peripheral.  


# Writing two Interrupt Service Routines (ISRs)

An ISR is written to service an event from a particular peripheral.  This is achieved by using a special macro function declaration.  In our case, we are using pins which carry the definition INT6 and PCINT0 - where the encoders have been wired too on the 3Pi+ - and which we can find are Pin Change enabled in the microcontroller manual.  You'll find them referenced in the setup routines setupEncoder0() and setupEncoder1().  Our ISR function declarations look like:

```C++
ISR( INT6_vect ) {
 // ....
 // ...ISR code - keep it short!...
 // ....
}
ISR( PCINT0_vect ) {
 // ....
 // ...ISR code - keep it short!...
 // ....
}
```


After calling the setup routines, these two functions will execute whenever the encoder changes state.  Note that the functions do not take any arguments or provide any return type.  To have any persistent data, these ISR's will have to operate on **global** variables.  Because we don't know when the ISR's will execute (e.g., can we predict when the wheel will rotate?), we also need to declare the associated global variables as **volatile**.   Volatile is a keyword to the compiler, informing the compiler not to use any cache based optimatisations.  Therefore, in global scope, we declare the following global variables to be used by our ISR functions:

```C++
// Volatile Global variables used by Encoder ISR.
volatile long count_e1; // used by encoder to count the rotation
volatile bool oldE1_A;  // used by encoder to remember prior state of A
volatile bool oldE1_B;  // used by encoder to remember prior state of B

volatile long count_e0; // used by encoder to count the rotation
volatile bool oldE0_A;  // used by encoder to remember prior state of A
volatile bool oldE0_B;  // used by encoder to remember prior state of B
```


For the remaining implementation details for the Pin Change Interrupt ISR's, you can read the comments provided in the example code.



# Example Encoder Code

This section of the labsheet will 


We can consider that the system now registers two 2-bit states (A and B) - one at the current time $(t)$, and a prior at time $(t-1)$.  We can also think of this as needing to store 4-bits of information.  This is boolean logic, so the theoretical number of possible combinations is $2^4$, 16 possible states.

|Index| Channel B (t)  | Channel A (t)  |-| Channel B (t-1)  | Channel A (t-1)  | Notes  |
|:---:|:---:|:---:|-|:---:|:---:|:---|
| 0 | 0  | 0  |-|  0 |  0 |  (no change, invalid) | 
| 1 | 0  | 0  |-|  0 | 1  |   |
| 2 | 0  | 0  |-|  1 |  0 |  |
| 3 | 0  | 0  |-|  1 |  1 | (change x2, invalid) |
| 4 | 0  | 1  |-|  0 |  0 |  |
| 5 | 0  | 1  |-|  0 |  1 | (no change, invalid)|
| 6 | 0  | 1  |-|  1 |  0 | (change x2, invalid) |
| 7 | 0  | 1  |-|  1 |  1 |  |
| 8 | 1  | 0  |-|  0 |  0 |  |
| 9 | 1  | 0  |-|  0 |  1 |  (change x2, invalid)|
| 10 | 1  | 0  |-|  1 |  0 |  (no change, invalid)|
| 11 | 1  | 0  |-|  1 |  1 |  |
| 12 | 1  | 1  |-|  0 |  0 |  (change x2, invalid)|
| 13 | 1  | 1  |-|  0 |  1 |  |
| 14 | 1  | 1  |-|  1 |  0 |  |
| 15 | 1  | 1  |-|  1 |  1 | (no change, invalid) |

The above logic table illustrates all the theoretical states of the encoders.  However, the impossible states have been noted.  



Everytime the motor physically rotates, it will generate a new transition on the encoder.  Typically, encoders refer to this property as **Count Per Revolution (CPR)**, **Pulses Per Revolution (PPR)** or **Lines Per Revolution (LPR)**.  The higher the CPR, the more precisely  rotation can be determined. This is because we are essentially dividing a full rotation (360 degrees, or 2\*PI radians) by the CPR.  One thing that will effect this, is whether the encoder is placed before or after any gearing on the motor.  It may be that a full rotation of the encoder is only a fraction of a full rotation of the wheel.  This is the case for your Romi.  Your Romi has a 120:1 gearbox.  




This section of the labsheet will walk you through the process of writing code to use a quadrature encoder.  There are more efficient ways to write this code, but hopefully the following example code will de-mystify the process.  

When using a quadrature encoder we read two channels, A or B.  We are looking for a change in value, so we need to record a historic value and a new value.  Therefore, we can keep track of four variables, each being a 0 or 1. 
- Channel A (1 or 0) the last time we read it (old, historic).
- Channel A (1 or 0) the new or latest reading.
- Channel B (1 or 0) the last time we read it (old, historic).
- Channel B (1 or 0) the new or latest reading.

We can then write all these possible combinations of the A and B channels as a table of all possible values.  When we read our sensor, and compare it to prior readings, we can then check against this table of possible combinations to look up what the wheel rotation must be (a look-up table).  

Because the previous and new states will change, we refer to them as variables. In the table below, each variable is a column of the table.  With four variables changing between 0 and 1, we essentially have a 4 bit repesentation.  4 bits creates a table with 16 possible combinations of states.  We could re-arrange the rows of the table below into any order and it would still be all possible 16 combintions.  The table layout below follows a convention of tracking the combination of binary transitions across columns right to left. 

Notice that, when the possible combinations are laid out in this way, we can read their binary representation as a decimal value, marked on the left hand side of the table.  We will use this decimal value later to identify which combination of sensor read and prior readings equate to which row of our table.
```C++
//   (8)   (4)   (2)    (1)  << binary significance
//   new   new   old   old
//    A     B     A     B    
//   ----  ----  ----  ----   
//0   0      0    0      0   // All possible combinations
//1   0      0    0      1   // of sensor read channels 
//2   0      0    1      0   // A and B, and prior reads
//3   0      0    1      1   // of channels A and B.   
//4   0      1    0      0     
//5   0      1    0      1     
//6   0      1    1      0     
//7   0      1    1      1     
//8   1      0    0      0    
//9   1      0    0      1     
//10  1      0    1      0     
//11  1      0    1      1     
//12  1      1    0      0     
//13  1      1    0      1     
//14  1      1    1      0     
//15  1      1    1      1 
```

Looking at the above table, we can read the right two columns as the previous A B states, and look in the left two columns at the new states, and see what changed.  We only want to know when our wheels are moving.  Therefore, when there is no change between old and new states for either A and B, our encoder is not moving, and we can remove these states:

```C++
//   new   new   old   old
//    A     B     A     B  
//   ----  ----  ----  ----   
//0   0      0    0      0    (invalid, no movement)
//1   0      0    0      1    
//2   0      0    1      0     
//3   0      0    1      1      
//4   0      1    0      0     
//5   0      1    0      1     (invalid, no movement)
//6   0      1    1      0     
//7   0      1    1      1     
//8   1      0    0      0    
//9   1      0    0      1     
//10  1      0    1      0     (invalid, no movement)
//11  1      0    1      1     
//12  1      1    0      0     
//13  1      1    0      1     
//14  1      1    1      0     
//15  1      1    1      1     (invalid, no movement)

```


We can also remove the states where both A and B change at the same time.  If both A and B change at the same time, we don't know which way the encoder rotated.  On the Romi this should be impossible, because the A and B channels are out of phase.  There are occasions and systems when both A and B might change together, so this is only true for our robot:

```C++
//   new   new   old   old
//    A     B     A     B  
//   ----  ----  ----  ----   
//0   0      0    0      0    (invalid, no movement)
//1   0      0    0      1    
//2   0      0    1      0     
//3   0      0    1      1    (invalid, impossible change)
//4   0      1    0      0     
//5   0      1    0      1    (invalid, no movement)
//6   0      1    1      0    (invalid, impossible change)
//7   0      1    1      1     
//8   1      0    0      0    
//9   1      0    0      1    (invalid, impossible change)
//10  1      0    1      0    (invalid, no movement)
//11  1      0    1      1     
//12  1      1    0      0    (invalid, impossible change)
//13  1      1    0      1     
//14  1      1    1      0     
//15  1      1    1      1    (invalid, no movement)

```


We've now reduced our possible states down from 16 to 8 just by applying some logic. Using the simple rules stated above, we can  add which way we expect the encoder to be rotating given the transition of either A or B from the prior state.  For the most part, we can assign a clockwise (CW) rotation, and assume the other must be counter-clockwise (CCW):
```C++
//   new   new   old   old
//    A     B     A     B  
//   ----  ----  ----  ----   
//0   0      0    0      0    (invalid, no movement)
//1   0      0    0      1    +1 (CW?)
//2   0      0    1      0    -1 (CCW?) 
//3   0      0    1      1    (invalid, impossible change)
//4   0      1    0      0    -1 (CCW?) 
//5   0      1    0      1    (invalid, no movement)
//6   0      1    1      0    (invalid, impossible change)
//7   0      1    1      1    +1 (CW?) 
//8   1      0    0      0    +1 (CW?)
//9   1      0    0      1    (invalid, impossible change)
//10  1      0    1      0    (invalid, no movement)
//11  1      0    1      1    -1 (CCW?) 
//12  1      1    0      0    (invalid, impossible change)
//13  1      1    0      1    -1 (CCW?) 
//14  1      1    1      0    +1 (CW?)
//15  1      1    1      1    (invalid, no movement)

```


Notice that there are questions marks beside CW and CCW.  That is because in theory we can label them as CW, CCW, but it depends on which way around the encoders and motors are wired into the Romi.  If Channel A and B are reversed, the direction will be reversed also.  Make sure you check this with your Romi later.

We now have a complete look up table for our encoders.  That means the code should be as simple as reading the encoder pins, and then checking against the look-up table.  

If we look at the table above, we can imagine writing an if(){} statement to select the appropriate table row given our set of readings, something like:

```C++
    if( (old_b == 1) && (old_a == 0) && (new_b == 0) && (new_a == 0) ) {        // Row 1
        
    } else if( (old_b == 0) && (old_a == 1) && (new_b == 0) && (new_a == 0) ) { // Row 2
        
    } else if( ... ) { // ... etc

```
However, this is very long winded.  

As mentioned before, we can actually add up the binary significance of each column of our table to find the decimal value representative of the row, labelled on the left most side above.  To do this, we'll first create 4 variables to store each of the readings.  Then, to gain the row number we'll shift each variable relative to which column in the table it represents.  In so doing, the 4 binary values placed consecutively into a new variable, `state`, read to add up to the decimal value of our look up table.  You will see this in the following example code in this operation:

```C++
  // Create a bitwise representation of our states
  // We do this by shifting the boolean value up by
  // the appropriate number of bits, as per our table
  // header:
  // binary Sig.:    (8)      (4)     (2)     (1)
  // << shift   :   (bit3)  (bit2)  (bit1)  (bit0)
  // Column     :   New A,  New B,  Old A,  Old B.
  byte state = 0;                   
  state = state | ( newE1_A  << 3 );  // shifting for each colum
  state = state | ( newE1_B  << 2 );
  state = state | ( oldE1_A  << 1 );
  state = state | ( oldE1_B  << 0 );  
```


The above example may look quite strange if you are not familiar with bitwise operations (if so, take a look <a href="https://www.programiz.com/c-programming/bitwise-operators">here</a>).  The four `state = ` lines are building up a byte variable with our binary values in the first four bits.  We can essentially break down the following code :
- state = state | ( newE1_A << 3);

Such that:
- `( newE1_A << 3 )` means shift the variable `newE1_A` up by 3 bits (therefore, from first bit to the fourth bit, 3 'jumps')
- `state | ( newE1_A << 3 )` means, logically OR in the above, such that any currently set bits of `state` remain intact.
- `state = state | ( newE1_A << 3 )` is the assignment of the above into the `state` variable.

Therefore, if `newE1_A = 1`, `newE1_B = 0`, `oldE1_A = 1` and `oldE1_B = 1`, our bitwise shifting code would be assembled into the variable `state` to equate to (in binary) :

```C++
    1011 // Binary -> decimal value 11
```


By using the above, we can read _state_ as a decimal value, and we can now use a familar if(){} statement to select the appropriate action (entries are missing - you'll complete this table in the later tasks):

```C++
if( state == 1 ) {           // row 1 from table
    count_e1 = count_e1 + 1;
    
} else if( state == 2 ) {    // row 2 from table
    // ?
    
} else if( state == 4 ) {    // row 4 from table
    // ?    
    
} else if( state == 7 ) {
    // ?
    
} else if( state == 8 ) {
    // ?
    
} else if( ... ) { // ...etc
```

To complete our software, we now need to read the encoder pins for channel A and channel B, and then compare the values to our look up table.  When we find the matching row on the table, we will simply increase or decrease the count for that encoder. 


# Exercise 1: Encoders with Pin Change Interrupts

The following code template is _nearly_ complete:

### Task
- Correct the below example code by writing in the increment or decrement operators for count_e0 and count_e1 into the if(){} statement blocks within the ISR's.  


```C++
// These #defines act like a find-and-replace
// in your code, and make your code more readable.
// Note that there is no #define for E0_B:.
// it's a non-standard pin, check out setupEncoder0().
#define E1_A_PIN  7
#define E1_B_PIN  23
#define E0_A_PIN  26

// Volatile Global variables used by Encoder ISR.
volatile long count_e1; // used by encoder to count the rotation
volatile bool oldE1_A;  // used by encoder to remember prior state of A
volatile bool oldE1_B;  // used by encoder to remember prior state of B

volatile long count_e0; // used by encoder to count the rotation
volatile bool oldE0_A;  // used by encoder to remember prior state of A
volatile bool oldE0_B;  // used by encoder to remember prior state of B



// put your setup code here, to run once:
void setup() {

  // These two function set up the pin
  // change interrupts for the encoders.
  // If you want to know more, find them
  // at the end of this file.  
  setupEncoder0();
  setupEncoder1();


  // Initialise the Serial communication
  // so that we can inspect the values of
  // our encoder using the Monitor.
  Serial.begin( 9600 );
    
  // Wait for serial connection to establish
  delay(1000);
    
  // Flag up reset to Serial monitor
  Serial.println("*** RESET ***");
}



// put your main code here, to run repeatedly:
void loop() {

  // Output the count values for our encoders
  // with a comma seperation, which allows for
  // two lines to be drawn on the Plotter.
  //
  // NOTE: count_e0 and count_e1 values are now 
  //       automatically updated by the ISR when 
  //       the encoder pins change.  
  //
  Serial.print( count_e0 );
  Serial.print( ", ");
  Serial.println( count_e1 );

  // short delay so that our plotter graph keeps
  // some history.
  delay( 2 );
}

// This ISR handles just Encoder 1
// ISR to read the Encoder1 Channel A and B pins
// and then look up based on  transition what kind of
// rotation must have occured.
ISR( INT6_vect ) {
  // First, Read in the new state of the encoder pins.
  // Standard pins, so standard read functions.
  boolean newE1_B = digitalRead( E1_B_PIN );
  boolean newE1_A = digitalRead( E1_A_PIN );

  // Some clever electronics combines the
  // signals and this XOR restores the
  // true value.
  newE1_A ^= newE1_B;

  // Create a bitwise representation of our states
  // We do this by shifting the boolean value up by
  // the appropriate number of bits, as per our table
  // header:
  //
  // State :  (bit3)  (bit2)  (bit1)  (bit0)
  // State :  New A,  New B,  Old A,  Old B.
  byte state = 0;
  state = state | ( newE1_A  << 3 );
  state = state | ( newE1_B  << 2 );
  state = state | ( oldE1_A  << 1 );
  state = state | ( oldE1_B  << 0 );


  // This is an inefficient way of determining
  // the direction.  However it illustrates well
  // against the lecture slides.
  if( state == 1 ) {
      count_e1 = count_e1 + 1;  // cw?
  } else if( state == 2 ) {
      ?????
  } else if( state == 4 ) {
      ?????
  } else if( ????? ) {
      
  } // Continue to build this table.

  // Save current state as old state for next call.
  oldE1_A = newE1_A;
  oldE1_B = newE1_B;

}


// This ISR handles just Encoder 0
// ISR to read the Encoder0 Channel A and B pins
// and then look up based on  transition what kind of
// rotation must have occured.
ISR( PCINT0_vect ) {

  // First, Read in the new state of the encoder pins.

  // Mask for a specific pin from the port.
  // Non-standard pin, so we access the register
  // directly.  
  // Reading just PINE would give us a number
  // composed of all 8 bits.  We want only bit 2.
  // B00000100 masks out all but bit 2
  boolean newE0_B = PINE & (1<<PINE2);
  //boolean newE0_B = PINE & B00000100;  // Does same as above.

  // Standard read fro the other pin.
  boolean newE0_A = digitalRead( E0_A_PIN ); // 26 the same as A8

  // Some clever electronics combines the
  // signals and this XOR restores the 
  // true value.
  newE0_A ^= newE0_B;


  
  // Create a bitwise representation of our states
  // We do this by shifting the boolean value up by
  // the appropriate number of bits, as per our table
  // header:
  //
  // State :  (bit3)  (bit2)  (bit1)  (bit0)
  // State :  New A,  New B,  Old A,  Old B.
  byte state = 0;                   
  state = state | ( newE0_A  << 3 );
  state = state | ( newE0_B  << 2 );
  state = state | ( oldE0_A  << 1 );
  state = state | ( oldE0_B  << 0 );

  // This is an inefficient way of determining
  // the direction.  However it illustrates well
  // against the lecture slides.
  if( state == 1 ) {
      count_e0 = count_e0 + 1;  // cw?
  } else if( state == 2 ) {
      ?????
  } else if( state == 4 ) {
      ?????
  } else if( ????? ) {
      
  } // Continue to build this table.
     
  // Save current state as old state for next call.
  oldE0_A = newE0_A;
  oldE0_B = newE0_B; 
}





/*
   This setup routine enables interrupts for
   encoder1.  The interrupt is automatically
   triggered when one of the encoder pin changes.
   This is really convenient!  It means we don't
   have to check the encoder manually.
*/
void setupEncoder1() {

  // Initialise our count value to 0.
  count_e1 = 0;

  // Initialise the prior A & B signals
  // to zero, we don't know what they were.
  oldE1_A = 0;
  oldE1_B = 0;

  // Setup pins for encoder 1
  pinMode( E1_A_PIN, INPUT );
  pinMode( E1_B_PIN, INPUT );

  // Now to set up PE6 as an external interupt (INT6), which means it can
  // have its own dedicated ISR vector INT6_vector

  // Page 90, 11.1.3 External Interrupt Mask Register – EIMSK
  // Disable external interrupts for INT6 first
  // Set INT6 bit low, preserve other bits
  EIMSK = EIMSK & ~(1<<INT6);
  //EIMSK = EIMSK & B1011111; // Same as above.
  
  // Page 89, 11.1.2 External Interrupt Control Register B – EICRB
  // Used to set up INT6 interrupt
  EICRB |= ( 1 << ISC60 );  // using header file names, push 1 to bit ISC60
  //EICRB |= B00010000; // does same as above

  // Page 90, 11.1.4 External Interrupt Flag Register – EIFR
  // Setting a 1 in bit 6 (INTF6) clears the interrupt flag.
  EIFR |= ( 1 << INTF6 );
  //EIFR |= B01000000;  // same as above

  // Now that we have set INT6 interrupt up, we can enable
  // the interrupt to happen
  // Page 90, 11.1.3 External Interrupt Mask Register – EIMSK
  // Disable external interrupts for INT6 first
  // Set INT6 bit high, preserve other bits
  EIMSK |= ( 1 << INT6 );
  //EIMSK |= B01000000; // Same as above

}

void setupEncoder0() {

    // Initialise our count value to 0.
    count_e0 = 0;

    // Initialise the prior A & B signals
    // to zero, we don't know what they were.
    oldE0_A = 0;
    oldE0_B = 0;

    // Setting up E0_PIN_B:
    // The Romi board uses the pin PE2 (port E, pin 2) which is
    // very unconventional.  It doesn't have a standard
    // arduino alias (like d6, or a5, for example).
    // We set it up here with direct register access
    // Writing a 0 to a DDR sets as input
    // DDRE = Data Direction Register (Port)E
    // We want pin PE2, which means bit 2 (counting from 0)
    // PE Register bits [ 7  6  5  4  3  2  1  0 ]
    // Binary mask      [ 1  1  1  1  1  0  1  1 ]
    //    
    // By performing an & here, the 0 sets low, all 1's preserve
    // any previous state.
    DDRE = DDRE & ~(1<<DDE6);
    //DDRE = DDRE & B11111011; // Same as above. 

    // We need to enable the pull up resistor for the pin
    // To do this, once a pin is set to input (as above)
    // You write a 1 to the bit in the output register
    PORTE = PORTE | (1<< PORTE2 );
    //PORTE = PORTE | 0B00000100;

    // Encoder0 uses conventional pin 26
    pinMode( E0_A_PIN, INPUT );
    digitalWrite( E0_A_PIN, HIGH ); // Encoder 0 xor

    // Enable pin-change interrupt on A8 (PB4) for encoder0, and disable other
    // pin-change interrupts.
    // Note, this register will normally create an interrupt a change to any pins
    // on the port, but we use PCMSK0 to set it only for PCINT4 which is A8 (PB4)
    // When we set these registers, the compiler will now look for a routine called
    // ISR( PCINT0_vect ) when it detects a change on the pin.  PCINT0 seems like a
    // mismatch to PCINT4, however there is only the one vector servicing a change
    // to all PCINT0->7 pins.
    // See Manual 11.1.5 Pin Change Interrupt Control Register - PCICR
    
    // Page 91, 11.1.5, Pin Change Interrupt Control Register 
    // Disable interrupt first
    PCICR = PCICR & ~( 1 << PCIE0 );
    // PCICR &= B11111110;  // Same as above
    
    // 11.1.7 Pin Change Mask Register 0 – PCMSK0
    PCMSK0 |= (1 << PCINT4);
    
    // Page 91, 11.1.6 Pin Change Interrupt Flag Register – PCIFR
    PCIFR |= (1 << PCIF0);  // Clear its interrupt flag by writing a 1.

    // Enable
    PCICR |= (1 << PCIE0);
}
```

# Exercise 2: Validating it works properly


- Download the code and use the Plotter to check if the values of count_e1 and count_e0 go up and down, depending on which way you rotate the wheel.  


- Rename the variables in the example code so that they mean more to you.  **This will help you in future lab sheets.** You may wish to consider:
    - Is encoder E1 the left or right motor? 
    - Is encoder E0 the left of right motor?
    - Which way would you like your robot to drive forwards? 
    - When the value of count_e1 or count_e0 goes up, does this represent foward or clockwise motion?

# Exercise 3: Exploring Utility (optional)

- Add your own code so that your robot will drive forward for +2000 encoder counts on both wheels and then stop.
    - Improve your code to get your robot to stop as near as possible on a count of +2000 for both wheels.  What are the main factors which make this difficult?
    - Improve your code further to get your robot to draw back to a count of 0 on both left and right wheels.  What are the main factors which make this difficult?
    
- Add your own code so that your robot can turn 90 degrees. 


  

# Exercise 4: Understanding Operation (optional)

- Determine how many encoder counts are required on both wheels to drive forward 10cm.
    - Complete this exercise experimentally with your robot (try and find it using a systematic method - e.g. calibrate!).  
    - Calculate the number of encoder counts that this should be, based on the gear box, the encoder counts per revolution, and the wheel radius.  The following have the necessary information you will need:
        - https://www.pololu.com/product/3542
        - https://www.pololu.com/product/3675/specs
    - Does your calculated encoder counts per wheel revolution match your calibrated value?
        
    
    
    

# Exercise 5: Basic Speed Control (optional):

- Create variables to store the **change of encoder counts** over time (or iteration) for the left and right motor respectively.
    - Have your code aim to maintain a fixed change of encoder counts by varying the power sent to the motor.
    - Test the performance your code by placing your robot on to different driving surfaces (e.g., not on a surface, carpet anda  table top).
  
  
- Evaluate this speed control to allow your Romi to drive in a straight line.  
    - Do you get better "straight line" performance than open loop control?
    - Do you get better "straight line" performance than controlling by strict encoder counts for left and right motors?
    